In [11]:
# ==============================================================================
# JUPYTER NOTEBOOK: NLP RESUME SKILL EXTRACTOR (FULL CODE)
# ==============================================================================

# ---------------------------------
# 1. ആവശ്യമായ ലൈബ്രറികൾ ഇറക്കുമതി ചെയ്യുക (Import Libraries)
# ---------------------------------
import PyPDF2
import spacy
import re
import os 
from spacy.matcher import Matcher # Matcher-ന്റെ ഉപയോഗം ഇവിടെയില്ല, പക്ഷെ മുമ്പ് ചേർത്തതുകൊണ്ട് നിലനിർത്തുന്നു.

# ---------------------------------
# 2. ടൂൾ സെറ്റപ്പും സ്കിൽ ലിസ്റ്റും (Tool Setup and Skill List)
# ---------------------------------

TECH_SKILLS = [
    "python", "java", "sql", "aws", "azure", "docker", "kubernetes", 
    "javascript", "html", "css", "mongodb", "react", "angular", "nlp", 
    "machine learning", "deep learning", "tableau", "power bi", "hadoop", "c++",
    "pandas", "numpy", "data analysis", "cloud computing"
]
SOFT_SKILLS = [
    "communication", "leadership", "teamwork", "problem solving", 
    "time management", "creativity", "adaptability", "mentoring", 
    "management", "agile", "scrum", "public speaking", "presentation"
]

# സ്കിൽ പര്യായങ്ങൾ (Synonyms) മാപ്പ് ചെയ്യാനുള്ള നിഘണ്ടു (Dictionary)
SKILL_MAPPING = {
    'analytical thinking': 'problem solving',
    'analytical skills': 'problem solving',
    'data visualization': 'tableau', 
    'nosql': 'mongodb', 
    'cloud services': 'aws', 
    'cloud platforms': 'aws', 
    'deep learning': 'machine learning', 
    'working with team': 'teamwork',
    'team player': 'teamwork',
    'public speaking': 'communication',
    'time organizing': 'time management',
    'group projects': 'teamwork', # Task 3 addition
    'business intelligence': 'power bi', # Task 3 addition
    'cloud computing': 'aws',        # JD ക്ക് വേണ്ട 'cloud computing' ને 'aws' ആയി മാപ്പ് ചെയ്യുന്നു 
    'strong communication': 'communication',
    'presentation skills': 'presentation',   # presentation സ്കിൽ മാപ്പ് ചെയ്യുന്നു
    'public speaking': 'presentation',
    'data analysis': 'data analysis' # ഒരു ഉറപ്പിനായി ഇത് ചേർക്കുന്നു
}

ALL_SKILLS = [s.lower() for s in TECH_SKILLS + SOFT_SKILLS]

# spaCy മോഡൽ ലോഡ് ചെയ്യുന്നു
try:
    nlp = spacy.load("en_core_web_sm")
    print("✅ spaCy model loaded successfully.")
except OSError as e:
    print(f"❌ Error loading spaCy model: {e}")

# ---------------------------------
# 3. PyPDF2 ഫംഗ്ഷൻ: Text Extract ചെയ്യുക
# ---------------------------------

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
            return text
    except Exception as e:
        print(f"❌ Error reading PDF file: {e}")
        return None

# ---------------------------------
# 4. spaCy ഫംഗ്ഷൻ: Skills വേർതിരിച്ചെടുക്കുക
# ---------------------------------

def extract_skills_from_text(text, skill_list):
    if not text:
        return set()
        
    processed_text = text.lower()
    doc = nlp(processed_text)
    
    # 1. RAW EXTRACTION: ആദ്യം കണ്ടെത്തുന്ന എല്ലാ സ്കില്ലുകളും ഇവിടെ ശേഖരിക്കുന്നു
    raw_found_skills = set() 
    
    # Noun Phrases, Single Words, Phrase Matching എന്നിവ ഉപയോഗിച്ച് കണ്ടെത്തുന്നു
    for chunk in doc.noun_chunks:
        chunk_text = chunk.text.lower()
        if chunk_text in skill_list:
            raw_found_skills.add(chunk_text)
            
    for token in doc:
        token_text = token.text.lower()
        if token_text in skill_list and len(token_text) > 2: 
            raw_found_skills.add(token_text)
            
    for skill in skill_list:
        if len(skill.split()) > 1:
            if skill in processed_text:
                raw_found_skills.add(skill)

    # 2. MAPPING: Synonyms ഉപയോഗിച്ച് മാസ്റ്റർ സ്കില്ലിലേക്ക് മാറ്റുന്നു
    final_mapped_skills = set()
    
    for skill in raw_found_skills:
        # global ആയി ഡിഫൈൻ ചെയ്ത SKILL_MAPPING ഉപയോഗിക്കുന്നു
        if skill in SKILL_MAPPING: 
            final_mapped_skills.add(SKILL_MAPPING[skill])
        else:
            final_mapped_skills.add(skill)
            
    return final_mapped_skills 

# ==============================================================================
# MAIN EXECUTION BLOCK
# ==============================================================================

RESUME_PATH = 'resume.pdf' 
JD_PATH = 'job_description.txt' 

if not os.path.exists(RESUME_PATH) or not os.path.exists(JD_PATH):
    print("🚨 ERROR: Resume or JD file not found in the current directory.")
else:
    print("✅ Files found. Starting extraction process.")
    
    # --- A. JD Process (Modified with encoding='utf-8') ---
    try:
        # പ്രധാന മാറ്റം: encoding='utf-8'
        with open(JD_PATH, 'r', encoding='utf-8') as f:
            jd_text = f.read()
    except Exception as e:
        print(f"❌ Error reading JD text: {e}")
        jd_text = ""

    # JD Skills വേർതിരിക്കുക
    jd_skills_required = extract_skills_from_text(jd_text, ALL_SKILLS)

    # --- B. Resume Process ---
    resume_text = extract_text_from_pdf(RESUME_PATH)
    resume_skills_got = extract_skills_from_text(resume_text, ALL_SKILLS)

    # --- C. Comparison and Report ---
    if resume_skills_got and jd_skills_required:
        gap_report = compare_skills(resume_skills_got, jd_skills_required)

        # Report പ്രിന്റ് ചെയ്യുക
        print("\n" + "="*45)
        print("          SKILL GAP ANALYSIS REPORT")
        print("="*45)
        print(f"📄 Candidate Resume Skills Found ({len(resume_skills_got)}):")
        print(f"   {sorted(list(resume_skills_got))}")
        print(f"💼 Job Description Required Skills ({len(jd_skills_required)}):")
        print(f"   {sorted(list(jd_skills_required))}\n")
        print("-" * 45)
        print(f"✔ MATCHING SKILLS ({len(gap_report['Matching Skills'])}):")
        print(f"   {sorted(list(gap_report['Matching Skills']))}")
        print(f"❌ MISSING SKILLS (GAP) ({len(gap_report['Missing Skills'])}):")
        print(f"   {sorted(list(gap_report['Missing Skills']))}")
        print(f"⭐ EXTRA SKILLS (Not required by JD) ({len(gap_report['Extra Skills'])}):")
        print(f"   {sorted(list(gap_report['Extra Skills']))}")
        print("="*45)
    else:
        print("\n❌ Could not extract enough skills to generate a report.")
        
# ==============================================================================


✅ spaCy model loaded successfully.
✅ Files found. Starting extraction process.

          SKILL GAP ANALYSIS REPORT
📄 Candidate Resume Skills Found (16):
   ['aws', 'communication', 'data analysis', 'hadoop', 'machine learning', 'management', 'numpy', 'pandas', 'power bi', 'presentation', 'problem solving', 'python', 'sql', 'tableau', 'teamwork', 'time management']
💼 Job Description Required Skills (13):
   ['aws', 'communication', 'data analysis', 'machine learning', 'management', 'power bi', 'presentation', 'problem solving', 'python', 'sql', 'tableau', 'teamwork', 'time management']

---------------------------------------------
✔ MATCHING SKILLS (13):
   ['aws', 'communication', 'data analysis', 'machine learning', 'management', 'power bi', 'presentation', 'problem solving', 'python', 'sql', 'tableau', 'teamwork', 'time management']
❌ MISSING SKILLS (GAP) (0):
   []
⭐ EXTRA SKILLS (Not required by JD) (3):
   ['hadoop', 'numpy', 'pandas']
